In [ ]:
!pip install ultralytics

import os
import yaml
import shutil
import random
import ultralytics
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [ ]:
random.seed(22520834)
path_data = Path("./signboard_dataset/train/images")
list_imgs_path = list(path_data.iterdir())
list_imgs_path.sort()

imgs_visualize = random.sample(list_imgs_path, 10)

for img_path in imgs_visualize:
    img = Image.open(img_path)
    width, height = img.size
    img.resize((200, 400), Image.Resampling.LANCZOS)

    label_path = img_path.parent.parent / 'labels' / (img_path.stem + '.txt')

    bounding_boxs = open(label_path, "r").readlines()
    for bbox in bounding_boxs:
        x, y, w, h = map(float, bbox.split()[1:])

        x_min = x - w / 2
        y_min = y - h / 2
        x_max = x + w / 2
        y_max = y + h / 2

        print(img_path, x_min, y_min, x_max, y_max)

        img1 = ImageDraw.Draw(img)
        img1.rectangle([(int(x_min * width), int(y_min * height)), (int(x_max * width), int(y_max * height))], outline="red", width=5)

    display(img)

In [ ]:
os.makedirs('./train/images', exist_ok=True)
os.makedirs('./train/labels', exist_ok=True)
os.makedirs('./val/images', exist_ok=True)
os.makedirs('./val/labels', exist_ok=True)

path_data = Path("./signboard_dataset/train/images")
list_imgs_path = list(path_data.iterdir())

len_val_data = len(list_imgs_path) // 5
list_imgs_path_val = random.sample(list_imgs_path, len_val_data)

for img_path in list_imgs_path:
    if img_path in list_imgs_path_val:
        shutil.copy(img_path, './val/images')
        label_path = img_path.parent.parent / 'labels1' / (img_path.stem + '.txt')
        if label_path.exists():
            shutil.copy(label_path, './val/labels')
    else:
        shutil.copy(img_path, './train/images')
        label_path = img_path.parent.parent / 'labels1' / (img_path.stem + '.txt')
        if label_path.exists():
            shutil.copy(label_path, './train/labels')

In [ ]:
dataset_info = {
    'train': './train',
    'val': './val',
    'nc': 1,
    'names': ['SignBoard'],
}

yamlfile_path = './dataset_signboard.yaml'

with open(yamlfile_path, 'w') as file:
    yaml.dump(dataset_info, file, default_flow_style=None)

!yolo train model=runs/detect/train13/weights/best.pt \
data=./dataset_signboard.yaml \
epochs=20 \
imgsz=1024 \
batch=8 \
weight_decay=0.4 \
dropout=0.2 \
patience=100 \
crop_fraction=1.0 \
erasing=0.1 \
copy_paste=0.25 \
mixup=0.2 \
mosaic=0.5 \
bgr=0.1 \
perspective=0.0001 \
shear=20\
multi_scale = True \
scale=0.1 \
translate=0.2 \
degrees=30 \
fliplr=0.5 \
flipud=0.1 \
hsv_v=0.8 \
hsv_s=0.8 \
hsv_h=0.8

In [ ]:
!yolo mode=predict model=/content/07.pt source="./test/images"
import glob
from IPython.display import Image, display

for image_path in glob.glob(f"./runs/detect/predict/*.jpg"):
  display(Image(filename=image_path, height = 640))
  print("\n")

In [ ]:
list_data_test = list(Path("./test/images").iterdir())
model = YOLO("./model.pt")

if Path("./answer.txt").exists():
  os.remove("./answer.txt")

answer_path = Path("./answer.txt")
answer_path.touch(exist_ok=True)

for img_path in tqdm(list_data_test, desc="Processing images"):
    result = model([img_path])[0]

    boxes = result.boxes
    img = Image.open(img_path)
    img_draw = ImageDraw.Draw(img)
    width, height = img.size

    with open(answer_path, "a") as file:
        for bbox in boxes:
            x, y, w, h = map(float, bbox.xywh[0].tolist())
            x /= width
            y /= height
            w /= width
            h /= height
            print(x, y, w, h)
            file.write(f"{img_path.stem} 0 {x} {y} {w} {h}\n")